In [29]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

class tomography_sim: 
    def __init__(self, lx=13, lz=11, N_detectors=24, velocity_free_=5.0, velocity_obst_=5.2):
        self.lx = lx
        self.lz = lz
        self.velocity_free = velocity_free_
        self.velocity_obst = velocity_obst_
        self.ndetectors = N_detectors

        self.v_lat = np.zeros((lx, lz))  # Lattice
        self.v_obst = np.zeros((lz, lx))  # Obstacles grid

    def generate_obstacles(self,shape= "rectangle"):
        """
        Generate obstacles in the grid
        """
        # Create a rectangular obstacle
        if shape == "rectangle":

            self.v_obst[1:9, 4:7] = 1
            return self.v_obst
        if shape == "delta":
            self.v_obst[2, 6] = 1

    def calculate_s(self, n_obst):
        """
        Calculate the anomaly for a given number of obstacles intersected
        """
        if n_obst > 1:
            return (n_obst - 1) * (1 / self.velocity_free - 1 / self.velocity_obst) * np.sqrt(2)
        else:
            return 0

    def calculate_anomalies(self):
        """
        Calculate arrival time anomalies for all detectors
        """
        anomalies = []
        for x in range(1, 13):  # Detectors x=1 to x=11
            # Each detector detects rays in diagonal directions
            anomalies_for_detector = []
            if x==1:
                n_obst_lr = self.count_obstacles_on_path(x, z_start=0, direction=1)
                anomaly_lr = self.calculate_s(n_obst_lr)
                anomalies_for_detector.append(anomaly_lr)
                # print(n_obst_lr,"hereee 1")
                # print(anomaly_lr,"hereee 1")
            elif x==12:
                # Right-to-left diagonal (1 slope)
                n_obst_rl = self.count_obstacles_on_path(x, z_start=0, direction=-1)
                anomaly_rl = self.calculate_s(n_obst_rl)
                # print(n_obst_rl,"hereee 12")
                # print(anomaly_rl,"hereee 12")

                anomalies_for_detector.append(anomaly_rl)
                

            else:
                # Left-to-right diagonal (-1 slope)
                n_obst_lr = self.count_obstacles_on_path(x, z_start=0, direction=1)
                anomaly_lr = self.calculate_s(n_obst_lr)
                anomalies_for_detector.append(anomaly_lr)

                # Right-to-left diagonal (1 slope)
                n_obst_rl = self.count_obstacles_on_path(x, z_start=0, direction=-1)
                anomaly_rl = self.calculate_s(n_obst_rl)
                anomalies_for_detector.append(anomaly_rl)


            anomalies.append(anomalies_for_detector)
        
        return anomalies

    def visualize(self, anomalies):
        """
        Visualize the obstacle field and ray paths
        """
        plt.figure(figsize=(10, 6))
        plt.xticks(range(0, 14)) 
        plt.yticks(range(0, 12))
        plt.imshow(self.v_obst, cmap="gray_r", origin="lower", extent=[0, self.lx, 0, self.lz])
        plt.colorbar(label="Obstacle Presence")
        plt.title("Obstacle Field and Detector Rays")
        plt.xlabel("X")
        plt.ylabel("Z")
        
        for x in range(1, 13):  # Detectors at x=1 to x=11
            # Plot left-to-right diagonal rays
            anomaly = anomalies[x - 1]

            if x==1:
                plt.plot([x, self.lx], [0, self.lx - x], "r--", label="Ray (L-R)" if x == 1 else "")
                plt.text(x, -1, f"{anomaly[0]:.2f}", color="red", fontsize=8, ha="center")



            elif x==12:
                plt.plot([x, 0], [0, x], "b--", label="Ray (R-L)" if x == 2 else "")
                plt.text(x, -1.5, f"{anomaly[0]:.2f}", color="blue", fontsize=8, ha="center")
                plt.grid(which='both', axis='both')


            else:
                plt.plot([x, 0], [0, x], "b--", label="Ray (R-L)" if x == 2 else "")
                plt.plot([x, self.lx], [0, self.lx - x], "r--", label="Ray (L-R)" if x == 1 else "")

                plt.text(x, -1, f"{anomaly[0]:.2f}", color="red", fontsize=8, ha="center")
                plt.text(x, -1.5, f"{anomaly[1]:.2f}", color="blue", fontsize=8, ha="center")


            
            # Annotate anomalies
            

            # if anomaly:
            #     # plt.text(x, 0.5, f"{sum(anomaly):.2f}", color="blue", fontsize=8, ha="center")

            #     plt.text(x, 0.5, f"{anomaly[0]:.2f}", color="red", fontsize=8, ha="center")


        plt.legend()
        plt.show()




    def count_obstacles_on_path(self, x_start, z_start, direction):
        """
        Count obstacles diagonally intersected along the ray path,
        considering the obstacle is represented at the top-left of its grid square.
        
        Args:
        - x_start (int): Starting x-coordinate (column).
        - z_start (int): Starting z-coordinate (row).
        - direction (int): +1 for right diagonal, -1 for left diagonal.
        
        Returns:
        - n_obst (int): Number of diagonally intersected obstacles.
        """
        n_obst = 0
        x, z = x_start, z_start
        lcl = True  # Flag to handle corner/edge transitions
        
        

        while 0 <= x < self.lx and 0 <= z < self.lz:
            # Check if the current cell contains an obstacle
            if self.v_obst[z, x] == 1 and direction == 1:
                n_obst += 1
            
            elif self.v_obst[z, x-1] == 1 and direction == -1:
                n_obst += 1
            
            # print(self.v_obst[z, x]) if x_start==7 and direction==-1 else True
            # Check the next diagonal cell
            next_x = x + direction
            next_z = z + 1

            if direction == -1 and 0 <= next_x  < self.lx and 0 <= next_z < self.lz:
                if self.v_obst[next_z, next_x-1] == 1 and lcl:
                    n_obst += 1
                    lcl = False  # Prevent double-counting

            elif direction == 1 and 0 <= next_x < self.lx and 0 <= next_z < self.lz:
                if self.v_obst[next_z, next_x] == 1 and lcl:
                    n_obst += 1
                    lcl = False

            # Reset `lcl` flag once moving past the transition
            # if not lcl and not (0 <= next_x < self.lx and 0 <= next_z < self.lz and self.v_obst[next_z, next_x] == 1):
            #     lcl = True

            # Move diagonally
            x += direction
            z += 1

        return n_obst
    
    def build_g_matrix(self):
        """
        Build the G matrix representing ray paths through the system
        """
        num_voxels = self.lx * self.lz
        G = np.zeros((self.ndetectors, num_voxels))  # Initialize G matrix
        detector_index = 0

        # Create a grid for visualization
        grid = np.zeros((self.lz, self.lx))


            
        for x_start in range(1, self.lx):
            # Left-to-right diagonal (\)
            x, z = x_start, 0
            while 0 <= x < self.lx and 0 <= z < self.lz:
                voxel_index = z * self.lx + x
                G[detector_index, voxel_index] = np.sqrt(2)  # Update G matrix
                grid[z, x] += np.sqrt(2)  # Update grid for visualization
                # self.visualize_ray(grid, f"Ray {detector_index + 1}: Left-to-right diagonal from x={x_start}")
                x += 1
                z += 1
            detector_index += 1

        for x_start in range(1, self.lx):


            # Right-to-left diagonal (/)
            x, z = x_start, 0
            while 0 <= x < self.lx and 0 <= z < self.lz:
                voxel_index = z * self.lx + x
                G[detector_index, voxel_index] = np.sqrt(2)  # Update G matrix
                grid[z, x] += np.sqrt(2)  # Update grid for visualization
                # self.visualize_ray(grid, f"Ray {detector_index + 1}: Right-to-left diagonal from x={x_start}")
                x -= 1
                z += 1
            detector_index += 1

        return G


In [30]:
sim = tomography_sim()
sim.generate_obstacles()
anomalies = sim.calculate_anomalies()
sim.visualize(anomalies)
# print(sim.v_obst)

<Figure size 1000x600 with 2 Axes>

In [31]:
print(anomalies)

[[0.03263569759322539], [0.03263569759322539, 0], [0.03263569759322539, 0], [0.02175713172881693, 0], [0.010878565864408465, 0], [0, 0.010878565864408465], [0, 0.02175713172881693], [0, 0.03263569759322539], [0, 0.03263569759322539], [0, 0.03263569759322539], [0, 0.03263569759322539], [0.03263569759322539]]


In [32]:
#adjust data to be regular
anomalies[0] = [anomalies[0][0],0]
anomalies[-1] = [0,anomalies[-1][0]]
print(anomalies)

[[0.03263569759322539, 0], [0.03263569759322539, 0], [0.03263569759322539, 0], [0.02175713172881693, 0], [0.010878565864408465, 0], [0, 0.010878565864408465], [0, 0.02175713172881693], [0, 0.03263569759322539], [0, 0.03263569759322539], [0, 0.03263569759322539], [0, 0.03263569759322539], [0, 0.03263569759322539]]


In [33]:
# flat tmat
flat_anomalies = []
for i in anomalies:
    flat_anomalies.append(i[0])
for i in anomalies:
    flat_anomalies.append(i[1])
print(flat_anomalies)
print("len of anomalies flat ->",len(flat_anomalies))

[0.03263569759322539, 0.03263569759322539, 0.03263569759322539, 0.02175713172881693, 0.010878565864408465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.010878565864408465, 0.02175713172881693, 0.03263569759322539, 0.03263569759322539, 0.03263569759322539, 0.03263569759322539, 0.03263569759322539]
len of anomalies flat -> 24


In [34]:
# noisy anomalies
random_variables = np.random.normal(loc=0, scale= 1/(18*np.sqrt(24))* np.linalg.norm(flat_anomalies),size=(24,1) ) # correct noise from Ioanna #24 is the size 
flat_random_variables = random_variables.flatten()
# print(flat_random_variables)
noisy_anom = flat_random_variables + flat_anomalies

In [35]:
g_matrix = sim.build_g_matrix()
print (g_matrix)

[[0.         1.41421356 0.         ... 0.         1.41421356 0.        ]
 [0.         0.         1.41421356 ... 0.         0.         1.41421356]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [36]:
def tikhonov(G, d_obs, epsilon):
    identity_matrix = np.identity(G.shape[1])
    x = np.linalg.inv(G.T @ G + epsilon*epsilon * identity_matrix) @ G.T @ d_obs
    return x

def function(g, vec_, t, epsilon):
    gs = np.matmul(g, vec_)
    # gs = gs.transpose()
    diff = np.linalg.norm(t - gs)
    this_sq = diff**2
    result = this_sq + epsilon**2 * np.linalg.norm(tikhonov(g,t,epsilon))**2 
    return result

In [37]:
epsilons = np.logspace(-7, 1, 1000, dtype = 'f')
# epsilons = np.linspace(1e-6,2,1000)
# epsilons = np.sqrt(epsilons)
min = 1.
epsilon_res = 100
list_l=[]

print(len(g_matrix))
print(len(g_matrix[0]))

noisy_vec = noisy_anom.tolist()
print(len(noisy_vec))


for e in epsilons:
    vec =  tikhonov(g_matrix, noisy_anom, e)
    lcl = function(g_matrix,vec,noisy_anom,e)
    list_l.append(lcl)
    if lcl < min:
        min = lcl
        epsilon_res = e


24
143
24


In [38]:
plt.plot(epsilons,list_l)
inversion_result_noisy = tikhonov(g_matrix, noisy_anom, epsilon_res)

print(epsilon_res,"hereeeeeeeeeeeeeeee")
plt.plot(epsilons, list_l, label='Data')
plt.xscale('log')  # Set x-axis to log scale

# Minimum point
min_x = epsilon_res
min_y =min
plt.axvline(x=min_x, color='r', linestyle='--', label=f'Minimum at x={min_x:.4f}')
plt.scatter(min_x, min_y, color='r')  # Marker for minimum point

# Legend
plt.legend()

# Show plot
plt.show()

1.846147e-05 hereeeeeeeeeeeeeeee


<Figure size 640x480 with 1 Axes>

In [39]:
m = inversion_result_noisy.reshape(11,13)

plt.figure(figsize=(10, 8))
sns.heatmap(m, cmap='inferno')  # Empty string for no numbers
plt.title('Heatmap of Matrix')
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
plt.show()

<Figure size 1000x800 with 2 Axes>

In [40]:
sim = tomography_sim()
sim.generate_obstacles("delta")
anomalies = sim.calculate_anomalies()
sim.visualize(anomalies)

<Figure size 1000x600 with 2 Axes>

In [41]:
anomalies[0] = [anomalies[0][0],0]
anomalies[-1] = [0,anomalies[-1][0]]

flat_anomalies = []

for i in anomalies:
    flat_anomalies.append(i[0])
for i in anomalies:
    flat_anomalies.append(i[1])

epsilons = np.logspace(-7, 1, 10000, dtype = 'f')
# epsilons = np.linspace(1e-6,2,1000)

min = 1.
epsilon_res = 100 # variable to get a the ideal epsilon
list_l=[]

for e in epsilons:
    vec =  tikhonov(g_matrix, flat_anomalies, e)
    lcl = function(g_matrix,vec,flat_anomalies,e)
    list_l.append(lcl)
    if lcl < min:
        min = lcl
        epsilon_res = e    

plt.plot(epsilons,list_l)
inversion_result_noisy = tikhonov(g_matrix, flat_anomalies, epsilon_res)

print("epsilon used ->",epsilon_res)
plt.plot(epsilons, list_l, label='Data')
plt.xscale('log')  # Set x-axis to log scale

# Minimum point
min_x = epsilon_res
min_y =min
plt.axvline(x=min_x, color='r', linestyle='--', label=f'Minimum at x={min_x:.7f}')
plt.scatter(min_x, min_y, color='r')  # Marker for minimum point

# Legend
plt.legend()

# Show plot
plt.show()

epsilon used -> 1.3189005e-05


<Figure size 640x480 with 1 Axes>

In [42]:
m = inversion_result_noisy.reshape(11,13)

plt.figure(figsize=(10, 8))
sns.heatmap(m, cmap='plasma')  # Empty string for no numbers
plt.title('Heatmap of Matrix')
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
plt.show()

<Figure size 1000x800 with 2 Axes>